In [45]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [46]:
df = pd.read_csv('train.csv')
majority_class = df[df['Exited'] == 0]
minority_class = df[df['Exited'] == 1]
majority_downsampled = resample(majority_class, replace=False, n_samples=len(minority_class), random_state=42)
df = pd.concat([majority_downsampled, minority_class])
df['Exited'].value_counts()


Exited
0    34921
1    34921
Name: count, dtype: int64

In [47]:
df.drop(['id','CustomerId','Surname'],axis=1,inplace=True)
mapGeography={'France':0,'Spain':1,'Germany':2}
df['Geography']=df['Geography'].map(mapGeography)
mapGender={'Male':0,'Female':1}
df['Gender']=df['Gender'].map(mapGender)

In [48]:
X,y=df.drop('Exited',axis=1),df['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
rf=RandomForestClassifier(n_estimators=400,max_depth=None,random_state=42)
rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=400, random_state=42)

In [49]:
y_train_pred=rf.predict(X_train)
y_test_pred=rf.predict(X_test)
print('Train Accuracy: ',accuracy_score(y_train,y_train_pred))
print('Test Accuracy: ',accuracy_score(y_test,y_test_pred))

Train Accuracy:  0.9998210226764269
Test Accuracy:  0.7963347412126852


In [50]:
df_submission=pd.read_csv('test.csv')
id_submission=df_submission['id']
df_submission.drop(['id','CustomerId','Surname'],axis=1,inplace=True)
df_submission['Geography']=df_submission['Geography'].map(mapGeography)
df_submission['Gender']=df_submission['Gender'].map(mapGender)
y_pred=rf.predict(df_submission)
submission=pd.DataFrame({'id':id_submission,'Exited':y_pred})
submission.to_csv('submission.csv',index=False)